In [1]:
from pynq import Overlay
conv = Overlay("./hw5.bit")

/usr/local/lib/python3.6/dist-packages/pynq/overlay.py:299: UserWarning: Users will not get PARAMETERS / REGISTERS information through TCL files. HWH file is recommended.
  warnings.warn(message, UserWarning)


In [2]:
conv.ip_dict

{'axi_cdma_0': {'addr_range': 65536,
  'driver': pynq.overlay.DefaultIP,
  'fullpath': 'axi_cdma_0',
  'gpio': {},
  'interrupts': {},
  'mem_id': 'SEG_axi_cdma_0_Reg',
  'phys_addr': 2116026368,
  'state': None,
  'type': 'xilinx.com:ip:axi_cdma:4.1'}}

In [3]:
cdma_address = conv.ip_dict['axi_cdma_0']['phys_addr']
print("Physical address of cdma: 0x" + format(cdma_address, '02x'))

Physical address of cdma: 0x7e200000


In [4]:
#set zynq address
zynq_addr = 0X30000000

In [5]:
from pynq import MMIO
cdma = MMIO(cdma_address, 65536)
zynq = MMIO(zynq_addr, 16384)

In [6]:
#open input file
fp = open('input.hex', "r")
line = fp.readline()
addr = 0
while line: #read the contents line by line until the end of the file
    hex_int = int(line, 16)
    zynq.write(addr, hex_int)
    line = fp.readline()
    addr = addr + 4
fp.close()

In [7]:
#write data from ZYNQ to bram0
cdma.write(0x0, 0x4) #CDMACR (CDMA_control)
cdma.write(0x18, zynq_addr) #SA (Source Address)
cdma.write(0x20, 0xC0000000) #DA (Destination Address)
cdma.write(0x28, 3180) #BTT (the number of Bytes to be transmitted)

In [8]:
#write data from bram1 to ZYNQ
cdma.write(0x0, 0x4) #CDMACR (CDMA_control)
cdma.write(0x18, 0xC2000000) #SA (Source Address)
cdma.write(0x20, zynq_addr) #DA (Destination Address)
cdma.write(0x28, 2704) #BTT (the number of Bytes to be transmitted)

In [9]:
#write the output data to output.hex
fp = open('output.hex', "w")
addr = 0
while addr<=2700:
    hex_int = zynq.read(addr)
    fp.writelines(str('%08x' % hex_int).upper()+"\n")
    #print(('%08x' % hex_int).upper())
    addr=addr+4
fp.close()

In [10]:
#checking the contents of the output and comparing with the golden
f1=open("output.hex","r")
f2=open("golden.hex","r")
i=-1
j=1
for line1 in f1:
    for line2 in f2:
        i=i+1
        if line1!=line2:
            j=0
            print("Different at pixel", i, ":", line1, "expected", line2)
        #else:
            #print("Same at pixel",i)
        break
if j==1:
    print("CONGRATULATIONS!")
else:
    print("OOPS!")
f1.close()
f2.close()

CONGRATULATIONS!
